In [82]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [83]:
# URL = "https://www.encuentra24.com/"
# URL = "https://www.encuentra24.com/el-salvador-es/clasificados"
URL = "https://www.encuentra24.com/el-salvador-es/autos-usados?q=f_style.3|imagesonly.on|number.50"

In [84]:
driver = webdriver.Edge(executable_path="./utils/msedgedriver.exe")
driver.get(URL)

In [85]:
def select_country(country:str):
    country = f"/{country}-es/clasificados"
    path = f"//a[contains(@href, '{country}')]"
    print(path)
    return driver.find_element(by=By.XPATH, value=path)

In [86]:
def close_window():
    button = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [87]:
def choose_category(category:str):
    driver.find_element(by=By.XPATH, value=category).click()

In [88]:
def price():
    price:str = "//div[@class='ann-ad-tile__price']"

In [89]:
close_window()

### Container

In [90]:
# elements = driver.find_elements(by=By.XPATH, value="//div[@class='ann-ad-tile']")
elements = driver.find_elements(by=By.XPATH, value="//div[@class='ann-subcat-listing']")

### Prices

In [91]:
prices_obs = driver.find_elements(by=By.XPATH, value="//div[@class='ann-ad-tile__price']")
# [price.text for price in prices_obs]

### Description

In [92]:
def description(driver:object) -> object:
    ad:str = "//div[@class='ann-ad-tile__content']"
    descrip:str = "//div[@class='row-fluid']"
    driver.find_element(by=By.XPATH, value=ad).click()
    descriptions:object = driver.find_element(by=By.XPATH, value=descrip)
    return descriptions.text

In [103]:
def clean_text(text:str, character, new_char) -> str:
    return text.replace(character,new_char)
    # spaced =  text.splitlines()
    # return spaced.strip()

In [94]:
ded = description(driver)

In [97]:
ded

'Garantía: Como es visto no hay garantía\nFinanciamiento: no\nPrecio negociable: si\nColor : AZUL\n¡VEHÍCULO CON PLACAS LISTAS!\n\nNISSAN SENTRA SV 2018 $9,850.00.\n\nPara mayor información puede comunicarse al por medio de llamada o WhatsApp.\n\nEnlace pagina web:\n\n\n\nCARACTERÍSTICAS:\n\n- Arranca y rueda.\n- 60,323 KILOMETROS actuales.\n- Bolsas buenas.\n- Llave de aproximación.\n- Sunroof.\n- Halógenos.\n- Pantalla touch.\n- Cámara de retroceso.\n- Transmisión automática.\n- Motor 1.8 L de 4 CIL.\n- Rines Especiales.\n- Paquete electrico.\n- Mandos al timón.\n- Conexion bluetooth.\n- Full extras.\n\nPRECIO: $9, FUERA DE ADUANA.\n\nPARA MAYOR INFORMACIÓN COMUNICARSE AL TELÉFONO O WHATSAPP: / .\n\n#sedan #importaciondevehiculos #carros #ventadevehiculos #venta #importacion #ventacarros #Nissan #Sentra #sentrasv #nissansentrasv'

In [104]:
clean_text(ded,'\n','')

'Garantía: Como es visto no hay garantíaFinanciamiento: noPrecio negociable: siColor : AZUL¡VEHÍCULO CON PLACAS LISTAS!NISSAN SENTRA SV 2018 $9,850.00.Para mayor información puede comunicarse al por medio de llamada o WhatsApp.Enlace pagina web:CARACTERÍSTICAS:- Arranca y rueda.- 60,323 KILOMETROS actuales.- Bolsas buenas.- Llave de aproximación.- Sunroof.- Halógenos.- Pantalla touch.- Cámara de retroceso.- Transmisión automática.- Motor 1.8 L de 4 CIL.- Rines Especiales.- Paquete electrico.- Mandos al timón.- Conexion bluetooth.- Full extras.PRECIO: $9, FUERA DE ADUANA.PARA MAYOR INFORMACIÓN COMUNICARSE AL TELÉFONO O WHATSAPP: / .#sedan #importaciondevehiculos #carros #ventadevehiculos #venta #importacion #ventacarros #Nissan #Sentra #sentrasv #nissansentrasv'